In [ ]:
# Install helmet-assignment helper code
!pip install ../input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.score import NFLAssignmentScorer, check_submission
from helmet_assignment.features import add_track_features

In [ ]:
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
import numpy as np
import random
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

# loading data

In [ ]:
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'
train_tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
train_helmets = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
test_tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
test_helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

In [ ]:
train_tracking_df = add_track_features(train_tracking)
test_tracking_df = add_track_features(test_tracking)

In [ ]:
def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def merge_tracking_data(args):
    video_frame, df = args
    gameKey,playID,view,frame = video_frame.split('_')
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame]
    this_tracking['video_frame'] = video_frame
    this_tracking['frame'] = frame
    this_tracking['video_gameKey_playID'] = str(gameKey) +'_'+str(playID)+'_'+str(view)
    df['frame'] = frame
    df['video_gameKey_playID'] = str(gameKey) +'_'+str(playID)+'_'+str(view)
    return this_tracking[['video_frame','video_gameKey_playID','x','y','player','frame']],df

In [ ]:
global tracking 
tracking = train_tracking_df

In [ ]:
p = Pool(processes=4)
tracking_list = []
df_list = list(train_helmets.groupby('video_frame'))
train_helmets = []
with tqdm(total=len(df_list)) as pbar:
    for (tracking_df,helmets_df) in p.imap(merge_tracking_data, df_list):
        train_helmets.append(helmets_df)
        tracking_list.append(tracking_df)
        pbar.update(1)
p.close()
train_helmets = pd.concat(train_helmets)
train_helmets = train_helmets.reset_index(drop=True)
train_tracking = pd.concat(tracking_list)
train_tracking = train_tracking.reset_index(drop=True)

In [ ]:
train_helmets = train_helmets.sort_values(['video_gameKey_playID','frame']).reset_index(drop=True)
train_tracking = train_tracking.sort_values(['video_gameKey_playID','frame']).reset_index(drop=True)

In [ ]:
global tracking 
tracking = test_tracking_df

In [ ]:
p = Pool(processes=4)
tracking_list = []
df_list = list(test_helmets.groupby('video_frame'))
test_helmets = []
with tqdm(total=len(df_list)) as pbar:
    for (tracking_df,helemts_df) in p.imap(merge_tracking_data, df_list):
        test_helmets.append(helemts_df)
        tracking_list.append(tracking_df)
        pbar.update(1)
p.close()
test_helmets = pd.concat(test_helmets)
test_helmets = test_helmets.reset_index(drop=True)
test_tracking = pd.concat(tracking_list)
test_tracking = test_tracking.reset_index(drop=True)

In [ ]:
test_helmets = test_helmets.sort_values(['video_gameKey_playID','frame']).reset_index(drop=True)
test_tracking = test_tracking.sort_values(['video_gameKey_playID','frame']).reset_index(drop=True)

# creating cnn lstm network

In [ ]:
training_percentage = 0.7
training_item_count = int(len(train_helmets['video_gameKey_playID'].unique())*training_percentage)
validation_item_count = len(train_helmets['video_gameKey_playID'].unique())-int(len(train_helmets['video_gameKey_playID'].unique())*training_percentage)
training_set = train_helmets['video_gameKey_playID'].unique()[:training_item_count]
validation_set = train_helmets['video_gameKey_playID'].unique()[-validation_item_count:]
training_set = train_helmets[train_helmets['video_gameKey_playID'].isin(training_set)]['video_frame'].unique()
validation_set = train_helmets[train_helmets['video_gameKey_playID'].isin(validation_set)]['video_frame'].unique()

In [ ]:
batch_size = 32
image_size =  60
input_shape = (image_size, image_size, 3)
dropout_rate = 0.4
classes_to_predict = sorted(train_helmets.label.unique())
class_weights = class_weight.compute_class_weight("balanced", classes_to_predict, train_helmets.label.values)
class_weights = dict(enumerate(class_weights))
n_classes = len(class_weights)

In [ ]:
from keras.layers import Bidirectional
from keras.layers import BatchNormalization

model = Sequential()
model.add(TimeDistributed(Conv2D(10, (2,2), activation= 'relu'),input_shape=(None,image_size,image_size,3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(20, (2,2), activation= 'relu')))
model.add(TimeDistributed(Conv2D(20, (1,1), activation= 'relu')))
model.add(TimeDistributed(Conv2D(20, (1,1), activation= 'relu')))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(n_classes, activation= 'softmax'))
model.compile(loss="categorical_crossentropy" , optimizer= 'adam' , metrics=['acc'])
model.summary()

In [ ]:
output_mapping_dict = {x: n for n,x in enumerate(train_helmets.label.unique())}

In [ ]:
global gameKey_playID_frame,helmets_data,tracking_data,time_steps,image_size,chosen_player,output_mapping,train
def data_preprocessing(args):
    _ ,video_frames = args
    video_frames = video_frames.values[0]
    images_batch = []
    labels = []
    for video_frame in video_frames:
        frame_number =  helmets_data[helmets_data.video_frame == str(video_frame)]['frame'].iloc[0] - 1 #frame number form zero
        frame_helmets = helmets_data[helmets_data.video_frame == str(video_frame)].reset_index(drop=True)
        frame_tracking = tracking_data[tracking_data['video_frame']==video_frame].reset_index(drop=True) 
        range_of_gameKey_playID = helmets_data[helmets_data.video_gameKey_playID == str(frame_helmets.video_gameKey_playID.iloc[0])]['video_frame'].unique()
        start_gameKey_playID_frame = 0 # number of first frame in gameKey_playID
        end_gameKey_playID_frame = len(range_of_gameKey_playID) - 1 # number of last frame in gameKey_playID
        if (time_steps * 60 + frame_number) > end_gameKey_playID_frame:
            diff = (time_steps * 60 + frame_number) - end_gameKey_playID_frame
            start = frame_number - diff
            end = end_gameKey_playID_frame
        else:
            start = frame_number
            end = time_steps * 60 + frame_number
            
        frame_helmets_images = [] 
        image = np.zeros((image_size,image_size))
        original_width = 1280
        original_height = 720
        x_max = 120
        y_max = 53.3
        frame_images = []
        new_width = np.floor((image_size-1) * (frame_helmets['left'].values + frame_helmets['width'].values / 2) / original_width)
        new_height = np.floor((image_size-1) * (frame_helmets['top'].values + frame_helmets['height'].values / 2) / original_height)
        new_width = np.where(new_width > (image_size-1) , (image_size-1) , new_width)
        new_height = np.where(new_height > (image_size-1) , (image_size-1) , new_height)
        frame_helmets_locations = np.concatenate((new_width.reshape(-1,1),new_height.reshape(-1,1)), axis=1)
        if chosen_player >= len(frame_helmets_locations):
            chosen_player_number = random.choice(range(frame_helmets['left'].values.shape[0]))
            chosen_player_location = frame_helmets_locations[chosen_player_number]
        else:
            chosen_player_number = chosen_player
            chosen_player_location = frame_helmets_locations[chosen_player]
        image[chosen_player_location.astype(int)] = 1
        frame_images.append(image)
        image[(new_width.astype(int),new_height.astype(int))] = 1
        frame_images.append(image)
        frame_tracking_players = frame_tracking
        new_x = np.floor((image_size-1) * (frame_tracking_players['x'].values) / x_max)
        new_y = np.floor((image_size-1) * (frame_tracking_players['y'].values) / y_max)
        new_x = np.where(new_x > (image_size-1) , (image_size-1) , new_x)
        new_y = np.where(new_y > (image_size-1) , (image_size-1) , new_y)
        new_y = new_y * -1
        tracking_labels = [output_mapping[x] for x in frame_tracking_players.player.values]
        image[(new_x.astype(int),new_y.astype(int))] = tracking_labels
        frame_images.append(image) 
        time_steps_data = []
    #    time_steps_data.append(np.array(images).reshape(image_size,image_size,3))
        tracking_step_image = np.zeros((image_size,image_size))
        helmets_step_image = np.zeros((image_size,image_size))
        for step in range(time_steps):
            images = []
            image = np.zeros((image_size,image_size))
            image[chosen_player_location.astype(int)] = 1
            images.append(image)
            frame_helmets_time_steps = helmets_data[helmets_data.video_frame == range_of_gameKey_playID[start+(step)*60]]
            new_width = np.floor((image_size-1) * (frame_helmets_time_steps['left'].values + frame_helmets_time_steps['width'].values / 2) / original_width) 
            new_height = np.floor((image_size-1) * (frame_helmets_time_steps['top'].values + frame_helmets_time_steps['height'].values / 2) / original_height) 
            new_width = np.where(new_width > (image_size-1) , (image_size-1) , new_width)
            new_height = np.where(new_height > (image_size-1) , (image_size-1) , new_height)
            helmets_step_image[(new_width.astype(int),new_height.astype(int))] = 1
            images.append(helmets_step_image)
            frame_tracking_time_steps = tracking_data[tracking_data.video_frame == range_of_gameKey_playID[start+(step)*60]]
            new_x = np.floor((image_size-1) * (frame_tracking_time_steps['x'].values) / x_max)
            new_y = np.floor((image_size-1) * (frame_tracking_time_steps['y'].values) / y_max)
            new_x = np.where(new_x > (image_size-1) , (image_size-1) , new_x)
            new_y = np.where(new_y > (image_size-1) , (image_size-1) , new_y)
            new_y = new_y * -1
            tracking_labels = [output_mapping[x] for x in frame_tracking_time_steps.player.values]
            tracking_step_image[(new_x.astype(int),new_y.astype(int))] = tracking_labels 
            images.append(tracking_step_image)
            time_steps_data.append(np.array(images).reshape(image_size,image_size,3))
            
        images_batch.append(time_steps_data)
        if train == True:    
            label = np.zeros((196,))
            label[output_mapping[frame_helmets['label'].values[chosen_player_number]]] = 1
            labels.append(label)

    if train == True:
        del images,tracking_labels,time_steps_data,frame_tracking_time_steps,label,frame_helmets_time_steps,tracking_step_image,new_x,new_y,new_width,new_height
        return np.array(images_batch),np.array(labels)
    else:
        del images,tracking_labels,time_steps_data,frame_tracking_time_steps,frame_helmets_time_steps,tracking_step_image,new_x,new_y,new_width,new_height
        return np.array(images_batch)

In [ ]:
gameKey_playID_frame,helmets_data,tracking_data,time_steps,image_size,chosen_player,output_mapping,train = training_set,train_helmets,train_tracking,6,60,1,output_mapping_dict,True
batch_size = 32
epochs = 25
p = Pool(processes=4)
chosen_player = 0
for epoch in range(epochs):
    if chosen_player > 22:
        chosen_player = 0
    training_frames = [i for n,i in enumerate(training_set)  if n % (random.choice(range(60))+1) == 0]
    random.shuffle(training_frames)
    for i in range(10):
        batch = []
        labels = []
        video_frames = training_frames[i*batch_size:i*batch_size+batch_size]
        video_frames_df = pd.DataFrame(video_frames,columns=['video_frames'])
        list_video_frames_df = video_frames_df.groupby(video_frames)
        for (sample,label) in p.imap(data_preprocessing, list_video_frames_df):        
            batch.append(sample[0])
            labels.append(label[0])
        batch = np.array(batch)
        labels = np.array(labels).reshape((-1,196))
        model.fit(batch,labels,batch_size=batch_size,class_weight = class_weights)
    chosen_player = chosen_player + 1
    print(f'epoch = {epoch}')

In [ ]:
model.save_weights('./model.hdf5')